In [ ]:
import os
import tempfile
import unittest
import filecmp
import sbol3
import paml
import tyto
import uml
import json

import paml_autoprotocol.plate_coordinates as pc
from paml_autoprotocol.execution_processor import ExecutionProcessor
from paml_autoprotocol.autoprotocol_specialization import AutoprotocolSpecialization, ResolvedTerm

from autoprotocol.protocol import \
    Spectrophotometry, \
    Protocol, \
    WellGroup, \
    Unit
from autoprotocol import container_type as ctype
from paml_autoprotocol.transcriptic_api import TranscripticAPI, TranscripticConfig
from paml.execution_engine import ExecutionEngine


%load_ext autoreload
%autoreload 2

In [ ]:
## Execute a PAML protocol, emitting an Autoprotocol protocol

# Setup PAML protocol for execution
out_dir = "."
doc = sbol3.Document()
sbol3.set_namespace('https://bbn.com/scratch/')
protocol_file = os.path.join(os.getcwd(), "../test/testfiles", "igem_ludox_test.nt")
doc.read(protocol_file, 'nt')
protocol = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018")
agent = sbol3.Agent("test_agent")


# Setup Autoprotocol specialization
out_dir = "out"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
autoprotocol_output = os.path.join(out_dir, "test_LUDOX_autoprotocol.json")
secrets_file = os.path.join(os.getcwd(), "../secrets/tx_secrets.json")
resolutions = {
    doc.find("https://bbn.com/scratch/LUDOX") : "rs1b6z2vgatkq7",
    doc.find("https://bbn.com/scratch/ddH2O") : "rs1c7pg8qs22dt"
}
api = TranscripticAPI(cfg=TranscripticConfig.from_file(secrets_file))
autoprotocol_specialization = AutoprotocolSpecialization(autoprotocol_output, api, resolutions)


# Execute the protocol
ee = ExecutionEngine(specializations=[autoprotocol_specialization])
parameter_values = [
    paml.ParameterValue(parameter=protocol.get_input("wavelength"), 
                        value=uml.LiteralIdentified(value=sbol3.Measure(100, tyto.OM.nanometer)))
]
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=parameter_values)


ee.specializations[0].protocol.instructions

In [ ]:
# Submit Autoprotocol object for execution at Strateos

tx = api.get_transcriptic_connection()
response = tx.submit_run(ee.specializations[0].protocol, 
                         project_id=api.cfg.project_id,
                         test_mode=True)

response

# The following is Scratch space

In [ ]:
#tx.inventory("96-flat")['results'][1]
#tx_protocols = tx.get_protocols()

In [ ]:
api._protocol_name_map["MakeContainers"].protocol

In [ ]:
 container_spec = {
                "name": "test create PAML container",
                "cont_type": ctype.FLAT96.shortname, # resolve with spec here
                "volume": "1000:microliter", # FIXME where does this come from?
                "properties": [
                    {
                        "key": "concentration",
                        "value": "10:millimolar"
                    }
                ]
            }
api.make_containers([container_spec])

In [ ]:
#[n.identity for n in protocol.nodes if isinstance(n, uml.CallBehaviorAction)]
provision = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018/CallBehaviorAction3")
parameters = [x.property_value for x in provision.behavior.lookup().parameters]
[(x., x.name) for x in parameters]

In [ ]:
import requests
resource = doc.find("https://bbn.com/scratch/LUDOX")
types = resource.types
resolved_types = [requests.get(t).headers['content-type'] for t in types]
resolved_types

In [ ]:
requests.request("https://identifiers.org/pubchem.substance:24866361")

In [ ]:
print(resource.type)
api.resolve_resource(resource)

In [ ]:
tx = api.get_transcriptic_connection()
[result] = tx.resources("ddh2o")["results"]
water_id = result["id"]
[result] = tx.resources("ludox")["results"]
ludox_id = result["id"]

resolutions = {specialization.unresolved_terms[0] : 'abc',
               specialization.unresolved_terms[1] : water_id,
               specialization.unresolved_terms[2] : ludox_id}
resolved_terms = [ResolvedTerm(x, resolutions[x]) for x in specialization.unresolved_terms]
resolved_terms

In [ ]:
print([x.unresolved_term.step for x in resolved_terms])
specialization.resolve(resolved_terms)


In [ ]:
import pubchempy
import pandas as pd
substance = "24901740"
s = pubchempy.Substance.from_sid(substance)


results = api.get_transcriptic_connection().resources(s.synonyms[0].split(" ")[0])
{r['id'] : (r['name'], r['pub_chem_id']) for r in results['results'] if 'pub_chem_id' in r}
#{r['id'] : list(r.keys()) for r in results['results']}
set([k for r in results['results'] for k in r.keys()])

results
#df = pd.DataFrame({r['id'] : [list(r.keys())] for r in results['results']})
#df

#import requests
#r = requests.get("https://identifiers.org/pubchem.substance:24901740")
#r.content